<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
    MySQL and MariaDB for Python Developers
</h1>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:55%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:62%; left:10%;">
    Data Scientist
</h3>
</div>

# MySQL Data Definition Language

Various RDBMSs implement languages close to official SQL specifications, but rarely either a strict superset nor a strict subset of actual specs.  It is complicated further by the fact that SQL specs include SQL-86, SQL-89, SQL-92, SQL:1999, SQL:2003, SQL:2006, SQL:2008, SQL:2011, and SQL:2016.  Each of these are different, not only by adding features but also sometimes deprecating them or changing spelling.

In other words, while most data definition commands you will use in MySQL are mostly similar to the equivalent commands in other RDBMSs, the equivalence is not precise.  MySQL supports a large number of data definition commands (all SQL-like); this lesson will highlight some particularly commonly used ones.

## CREATE, DROP, ALTER

The basic verbs for working with *data* in MySQL are SELECT, INSERT, UPDATE, DELETE, and perhaps TRUNCATE.  The first of those does not actually change data, just retrieve it.  But the verbs of the MySQL data definitions language (DDL) are CREATE, DROP, and ALTER.  GRANT is borderline, but possibly worth including as DDL.  These affect the *shape* of data rather than the content as such; however, many changes to shape by implication change underlying data globally (such as by removing it, or by changing its data type).

Each regular DDL verb is followed by a noun, and usually by some additional qualifiers.  For example, we can DROP the following objects:


| Objects       | Objects       | Objects      
|:--------------|:--------------|:--------------
| DATABASE      | EVENT         | FUNCTION
| FUNCTION UDF  | INDEX         | PREPARE
| PROCEDURE     | RESOURCE GROUP| ROLE
| SCHEMA        | SERVER        | SPATIAL REFERENCE SYSTEM
| TABLE         | TABLESPACE    | TRIGGER
| USER          | VIEW

Most of those same things can be CREATE'd or ALTER'd.  All of these are useful, of course, but USER/ROLE, TABLE, DATABASE, and INDEX are those you will use during typical database configuration.

Although the steps in this lesson are ones you would typically perform in an administration interface, let us work within the notebook, and using a `mysql.connector` connection.

In [1]:
import mysql.connector 
user = 'ine_student'
pwd = 'ine-password'
host = 'localhost'
port = '3306'
db = 'ine'

conn = mysql.connector.connect(database=db, host=host, user=user, password=pwd, port=port)
cur = conn.cursor()

## Refactoring tables

Let us recall the definition of the `census_zipcode_geography` table.  It has some flaws we can improve.  We might define a simple function to view table definitions from Python.

In [2]:
def table_schema(table_name):
    import pandas as pd
    cur.execute(f"SHOW columns FROM {table_name}")
    info_cols = [c[0] for c in cur.description]
    schema = cur.fetchall()
    df = pd.DataFrame(schema, columns=info_cols)
    # Cleaner to show DataFrame with str rather than bytes
    df['Type'] = df.Type.str.decode('utf-8')
    # And nullable as Bool value
    df['Null'] = df.Null == 'YES'
    return df

Viewing the table, we can see significant redundancy.  `awater` and `aland` express the same quantity as `awater_sqmi` and `aland_sqmi`, respectively, only in different units (square meters versus square miles).  Moreover, `location` is simply a combination of `intptlat` and `intptlong`.  Not only does this waste storage space, it could potentially lead to inconsistencies in the data.

In [3]:
table_schema('census_zipcode_geography')

,Field,Type,Null,Key,Default,Extra
0,USPS,char(5),False,PRI,None,
1,ALAND,bigint,True,,None,
2,AWATER,bigint,True,,None,
3,ALAND_SQMI,"decimal(8,3)",True,,None,
4,AWATER_SQMI,"decimal(8,3)",True,,None,
5,INTPTLAT,double,True,,None,
6,INTPTLONG,double,True,,None,
7,location,point,True,,None,


## Generated columns

Suppose we with to emulate initial `census_zipcode_geography` table, but enforce data integrity among columns.  We might define a new table as:

In [4]:
sql_geography = """
CREATE TABLE census_zipcode_integrity (
  zipcode CHAR(5),
  usps TEXT AS (zipcode),     -- Compatible with queries using old name
  aland BIGINT,               -- m^2 
  awater BIGINT,
  aland_sqmi DECIMAL(8, 3) AS (aland / 2589988.1103), 
  awater_sqmi DECIMAL(8, 3) AS (awater / 2589988.1103),
  lat DOUBLE AS (ST_X(location)),
  lon DOUBLE AS (ST_Y(location)),
  location POINT,              -- use geometric type for lat/lon
  PRIMARY KEY (zipcode)  
);
"""
cur.execute('DROP TABLE IF EXISTS census_zipcode_integrity')
cur.execute(sql_geography)
conn.commit()

Let us insert an invented data row to demonstrate.  Notice that the MySQL `POINT` type is a binary representation that we need special functions to parse into more usable numbers (as the generated columns do for us).

In [5]:
from pprint import pprint
sql_99999 = """
INSERT INTO census_zipcode_integrity (zipcode, aland, awater, location)
VALUES (99999, 25000000, 10000000, POINT (37.1, -95.7))
"""
try:
    cur.execute(sql_99999)
except Exception as err:
    # If we've already inserted it, just show message
    print(err)

cur.execute("SELECT * FROM census_zipcode_integrity WHERE zipcode='99999';")
for row in cur:
    pprint(dict(zip((c[0] for c in cur.description), row)))

# Don't actually keep this data
conn.rollback()

{'aland': 25000000,
 'aland_sqmi': Decimal('9.653'),
 'awater': 10000000,
 'awater_sqmi': Decimal('3.861'),
 'lat': 37.1,
 'location': bytearray(b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xcd\xcc\xcc'
                       b'\xcc\xcc\x8cB@\xcd\xcc\xcc\xcc\xcc\xecW\xc0'),
 'lon': -95.7,
 'usps': '99999',
 'zipcode': '99999'}


We can copy over the non-generated data into the new table.

In [6]:
sql_copy_zips = """
INSERT INTO census_zipcode_integrity (zipcode, aland, awater, location)
SELECT usps, aland, awater, location FROM census_zipcode_geography;
"""
cur.execute(sql_copy_zips)
conn.commit()

The census bureau provides the data as both square meters and square miles.  It appears that in 123 of the 33,144 rows, their conversion was not identical to ours.  Whether this matters is a domain judgement.

In [7]:
cur.execute("""
  SELECT new.zipcode, new.aland_sqmi as converted, old.aland_sqmi reported
  FROM census_zipcode_geography old
  JOIN census_zipcode_integrity new
  ON (old.usps = new.zipcode)
  WHERE new.aland_sqmi != old.aland_sqmi;""")

rows = cur.fetchall()
for row in rows[:8]:
    print(row)
print("Total rows:", len(rows))

('00685', Decimal('77.532'), Decimal('77.533'))
('03741', Decimal('76.932'), Decimal('76.933'))
('06074', Decimal('28.055'), Decimal('28.056'))
('13656', Decimal('85.074'), Decimal('85.075'))
('14454', Decimal('50.623'), Decimal('50.624'))
('20106', Decimal('60.225'), Decimal('60.226'))
('24018', Decimal('45.385'), Decimal('45.386'))
('24426', Decimal('332.864'), Decimal('332.865'))
Total rows: 123


## Altering tables

We have copied data from some tables into related tables to refactor the data organization.  We can also change tables in place.  This should be done with caution, since some such alterations will lose information.  In many cases, we can make alterations such as changing to a compatible data type or adding columns, without harming existing data.

Recall the simple table that records long descriptions of zipcode field keys:

In [8]:
conn = mysql.connector.connect(database=db, host=host, user=user, password=pwd, port=port)
cur = conn.cursor()
table_schema("census_zipcode_fields")

,Field,Type,Null,Key,Default,Extra
0,fieldname,varchar(15),False,PRI,None,
1,description,text,True,,None,


In [9]:
sql_new = """
CREATE TABLE zipcode_fields_altered
SELECT * FROM census_zipcode_fields;
"""
cur.execute("DROP TABLE IF EXISTS zipcode_fields_altered")
cur.execute(sql_new)
conn.commit()

In [10]:
sql_refcol = """
ALTER TABLE zipcode_fields_altered
ADD reference CHAR(50) DEFAULT 'https://www.census.gov/data/tables.html';
"""
cur.execute(sql_refcol)
conn.commit()

In [11]:
cur.execute('SELECT * FROM zipcode_fields_altered LIMIT 3;')
cur.fetchall()

[('ALAND',
  'Land Area (square meters) - Created for statistical purposes only',
  'https://www.census.gov/data/tables.html'),
 ('ALAND_SQMI',
  'Land Area (square miles) - Created for statistical purposes only',
  'https://www.census.gov/data/tables.html'),
 ('AWATER',
  'Water Area (square meters) - Created for statistical purposes only',
  'https://www.census.gov/data/tables.html')]

## Summary

We looked here at highlights of the MySQL DDL.  Various other elements of it are shown in other lessons, both before and after this one.

In the next lesson we look at data types available for MySQL.